In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project

/content/drive/My Drive/Colab Notebooks/Github/SindyAutoencoders_project


In [5]:
import os
import datetime
import pandas as pd
import numpy as np
from robi.example_LV import get_lofka_volterra_data
from src.sindy_utils import library_size
from src.training import train_network
import tensorflow as tf

# Generate data

In [6]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lofka_volterra_data(128, noise_strength=noise_strength)
validation_data = get_lofka_volterra_data(20, noise_strength=noise_strength)

In [ ]:
training_data

# Set up model and training parameters

In [7]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 2
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 1e-4
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [9]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'LV_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    # tf.reset_default_graph() didn't work anymore
    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.044791292399168015, (0.03991676, 47.659744, 0.9918648, 0.93744165)
   validation loss 0.045885439962148666, (0.04135255, 44.21105, 1.0241078, 0.93744165)
decoder loss ratio: 0.200807, decoder SINDy loss  ratio: 1.033422
Epoch 100
   training loss 2.1557159925578162e-05, (3.5287035e-06, 0.02589908, 0.009074572, 1.4531093)
   validation loss 2.7550062441150658e-05, (7.1992226e-06, 0.041294564, 0.01690291, 1.4531093)
decoder loss ratio: 0.000035, decoder SINDy loss  ratio: 0.017057
Epoch 200
   training loss 1.6197434888454154e-05, (1.6872351e-06, 0.015480702, 0.0033302717, 1.2629104)
   validation loss 2.037580998148769e-05, (5.0682224e-06, 0.020946154, 0.0058386857, 1.2629104)
decoder loss ratio: 0.000025, decoder SINDy loss  ratio: 0.005892
Epoch 300
   training loss 1.3130711522535421e-05, (1.018874e-06, 0.011609984, 0.002625069, 1.0688332)
   validation loss 1.6901427443372086e-05, (3.9893866e-06, 0.017598309, 0.0046387804, 1.0688332)


<ipython-input-9-49c4b7839fd5>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({**results_dict, **params}, ignore_index=True)
